# 🚢 EPA Titanic — Logistic Regression V1
Modelo de **Regresión Logística** con Feature Engineering y Preprocesamiento.  
El objetivo es comparar contra los experimentos previos con KNN.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# 📥 Celda 2 — Carga de datos
Leemos `train.csv` y `test.csv` desde `/kaggle/input/titanic/`.

In [2]:
import pandas as pd
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
test_df = pd.read_csv("/kaggle/input/titanic/test.csv")

print("Shape train:", train_df.shape)
print("Shape test :", test_df.shape)
train_df.head()


Shape train: (891, 12)
Shape test : (418, 11)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# 🔎 Celda 3 — EDA breve (faltantes y balance de Survived)
Revisamos columnas, valores faltantes (conteo y porcentaje) y el balance de la variable objetivo `Survived`.


In [3]:
import numpy as np  # importamos numpy para operaciones numéricas simples

# Mostramos la lista completa de columnas disponibles en el dataset de entrenamiento
print("Columnas en train:", train_df.columns.tolist())  # imprime los nombres de las columnas de train_df

# ---- Faltantes en TRAIN ----
na_train = train_df.isna().sum()  # cuenta cuántos valores NaN hay por columna en train
na_train_pct = (na_train / len(train_df)).round(3)  # calcula el porcentaje de NaN por columna (redondea a 3 decimales)
faltantes_train = pd.DataFrame({"nulos": na_train, "porcentaje": na_train_pct})  # arma una tabla con conteo y porcentaje
print("\nFaltantes en train (conteo y %):")  # título de sección para legibilidad
print(faltantes_train.sort_values("nulos", ascending=False))  # muestra primero las columnas con más faltantes

# ---- Faltantes en TEST ----
na_test = test_df.isna().sum()  # cuenta NaN por columna en test
na_test_pct = (na_test / len(test_df)).round(3)  # porcentaje de NaN por columna en test
faltantes_test = pd.DataFrame({"nulos": na_test, "porcentaje": na_test_pct})  # tabla similar para test
print("\nFaltantes en test (conteo y %):")  # título de sección
print(faltantes_test.sort_values("nulos", ascending=False))  # ordena de mayor a menor cantidad de nulos

# ---- Distribución de la variable objetivo ----
y_counts = train_df["Survived"].value_counts().sort_index()  # cuenta cuántos 0 y 1 hay (ordenado por valor)
y_ratio = train_df["Survived"].value_counts(normalize=True).sort_index().round(4)  # proporciones de 0 y 1 (redondeadas)

print("\nDistribución de Survived (conteo):")  # título de sección
print(y_counts)  # imprime conteos de 0 y 1

print("\nDistribución de Survived (proporción):")  # título de sección
print(y_ratio)  # imprime proporciones de 0 y 1


Columnas en train: ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']

Faltantes en train (conteo y %):
             nulos  porcentaje
Cabin          687       0.771
Age            177       0.199
Embarked         2       0.002
PassengerId      0       0.000
Survived         0       0.000
Pclass           0       0.000
Name             0       0.000
Sex              0       0.000
SibSp            0       0.000
Parch            0       0.000
Ticket           0       0.000
Fare             0       0.000

Faltantes en test (conteo y %):
             nulos  porcentaje
Cabin          327       0.782
Age             86       0.206
Fare             1       0.002
PassengerId      0       0.000
Pclass           0       0.000
Name             0       0.000
Sex              0       0.000
SibSp            0       0.000
Parch            0       0.000
Ticket           0       0.000
Embarked         0       0.000

Distribución de Survi

# 🧱 Celda 4 — Definición de variables base y objetivo
Separamos las columnas que usaremos como **features base** (`X`, `X_test`) y el **target** (`y`).  
Además guardamos `PassengerId` del test para construir luego el archivo de submission.


In [4]:
# --- Definición de columnas base ---
# Numéricas originales
base_num_features = ["Age", "SibSp", "Parch", "Fare", "Pclass"]

# Categóricas originales
base_cat_features = ["Sex", "Embarked"]

# --- Construcción de X (features de entrenamiento) ---
X = train_df[base_num_features + base_cat_features].copy()  # seleccionamos columnas base de train

# --- Target ---
y = train_df["Survived"].astype(int)  # convertimos Survived a entero (0/1)

# --- Construcción de X_test (features de predicción) ---
X_test = test_df[base_num_features + base_cat_features].copy()  # seleccionamos columnas base de test

# --- Guardamos PassengerId para el submission ---
test_passenger_id = test_df["PassengerId"].copy()  # IDs del set de test

# Vista previa de las primeras filas de X
X.head()


,Age,SibSp,Parch,Fare,Pclass,Sex,Embarked
0,22.0,1,0,7.2500,3,male,S
1,38.0,1,0,71.2833,1,female,C
2,26.0,0,0,7.9250,3,female,S
3,35.0,1,0,53.1000,1,female,S
4,35.0,0,0,8.0500,3,male,S


# 🧪 Celda 5 — Ingeniería de atributos
Creamos nuevas variables a partir de las columnas originales:  

- `FamilySize = SibSp + Parch + 1`  
- `IsAlone = 1 si FamilySize == 1, else 0`  
- `Title` (extraído de la columna `Name`)  
- `CabinKnown = 1 si Cabin no es NaN, else 0`  
- `FarePerPerson = Fare / FamilySize`  
- `TicketGroupSize = cantidad de pasajeros que comparten el mismo Ticket`  

Usamos un `FunctionTransformer` para integrar estas transformaciones dentro del `Pipeline`.


In [7]:
from sklearn.preprocessing import FunctionTransformer

# Definimos la función que añade nuevas features al DataFrame
def add_features(df):
    df = df.copy()  # trabajamos sobre una copia para no alterar el original

    # --- FamilySize e IsAlone ---
    df["FamilySize"] = df["SibSp"].fillna(0) + df["Parch"].fillna(0) + 1
    df["IsAlone"] = (df["FamilySize"] == 1).astype(int)

    # --- Title (extraído de Name) ---
    if "Name" in df.columns:
        # extrae el título que está entre la coma y el punto
        titles = df["Name"].str.extract(r",\s*([^\.]+)\.")[0]
        # unifica variantes
        titles = titles.replace({"Mlle": "Miss", "Ms": "Miss", "Mme": "Mrs"})
        # agrupa títulos raros en "Other"
        titles = titles.where(titles.isin(["Mr","Mrs","Miss","Master"]), "Other")
        df["Title"] = titles.fillna("Other")
    else:
        df["Title"] = "Other"

    # --- CabinKnown (1 si Cabin no es NaN) ---
    if "Cabin" in df.columns:
        df["CabinKnown"] = (~df["Cabin"].isna()).astype(int)
    else:
        df["CabinKnown"] = 0

    # --- FarePerPerson ---
    df["FarePerPerson"] = df["Fare"].fillna(df["Fare"].median()) / df["FamilySize"].replace(0, 1)

    # --- TicketGroupSize ---
    if "Ticket" in df.columns:
        counts = df["Ticket"].map(df["Ticket"].value_counts())
        df["TicketGroupSize"] = counts.fillna(1).astype(int)
    else:
        df["TicketGroupSize"] = 1

    return df

# Creamos el transformador para integrarlo en el pipeline
feature_engineering = FunctionTransformer(add_features, validate=False)

# --- Añadimos columnas auxiliares (Name, Ticket, Cabin) a X y X_test, necesarias para la ingeniería de features ---
for col in ["Name", "Ticket", "Cabin"]:
    if col not in X.columns:
        X[col] = train_df[col]
    if col not in X_test.columns:
        X_test[col] = test_df[col]

print("✅ Ingeniería de atributos lista (FunctionTransformer definido).")


✅ Ingeniería de atributos lista (FunctionTransformer definido).


# 🧽 Celda 6 — Preprocesamiento (imputación, escalado y One-Hot)
Definimos un `ColumnTransformer` que:
- Imputa numéricas con **mediana** y las **escala** (LogReg es sensible a magnitudes).
- Imputa categóricas con **moda** y aplica **One-Hot** (incluye `Title` creado en FE).


In [8]:
# Importamos utilidades para construir el preprocesamiento por tipo de columna
from sklearn.compose import ColumnTransformer            # aplicar transformadores por columnas
from sklearn.pipeline import Pipeline                    # encadenar pasos (FE -> preprocesamiento -> modelo)
from sklearn.impute import SimpleImputer                 # imputación de valores faltantes
from sklearn.preprocessing import StandardScaler, OneHotEncoder  # escalado y codificación one-hot

# Definimos las columnas que EXISTIRÁN DESPUÉS del paso de Feature Engineering (Celda 5)
# (Ojo: columnas auxiliares como Name/Ticket/Cabin NO van aquí; solo las features finales que alimentarán al modelo)
num_features = [
    "Age", "SibSp", "Parch", "Fare", "Pclass",     # numéricas originales
    "FamilySize", "IsAlone", "FarePerPerson",      # numéricas creadas
    "TicketGroupSize", "CabinKnown"                # numéricas/bool creadas
]

cat_features = [
    "Sex", "Embarked", "Title"                     # categóricas (Title viene de FE)
]

# Pipeline para columnas numéricas: imputar con mediana + escalar (recomendado para LogReg)
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),   # rellena NaN con la mediana (robusta a outliers)
    ("scaler", StandardScaler())                     # estandariza a media 0 y desviación 1
])

# Pipeline para columnas categóricas: imputar con la moda + One-Hot encoder
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),              # rellena NaN con el valor más frecuente
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))  # dummies densos; ignora categorías nuevas
])

# Combinamos ambos pipelines por tipo de columna
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),   # aplica numeric_transformer a num_features
        ("cat", categorical_transformer, cat_features) # aplica categorical_transformer a cat_features
    ],
    remainder="drop"  # descarta columnas no especificadas (p. ej., Name/Ticket/Cabin auxiliares)
)

# Mensaje de control para verificar listas
print("✅ Preprocesamiento definido.")
print("Num features:", num_features)
print("Cat features:", cat_features)


✅ Preprocesamiento definido.
Num features: ['Age', 'SibSp', 'Parch', 'Fare', 'Pclass', 'FamilySize', 'IsAlone', 'FarePerPerson', 'TicketGroupSize', 'CabinKnown']
Cat features: ['Sex', 'Embarked', 'Title']


# 🤖 Celda 7 — Pipeline con Regresión Logística y validación cruzada
Encadenamos **Feature Engineering → Preprocesamiento → LogisticRegression** en un `Pipeline`
y evaluamos con **validación cruzada estratificada (5 folds)** usando *accuracy*.


In [9]:
# Importamos el modelo y utilidades de validación
from sklearn.linear_model import LogisticRegression                  # modelo de regresión logística
from sklearn.model_selection import StratifiedKFold, cross_val_score  # CV estratificada y evaluación
import numpy as np                                                    # operaciones numéricas

# Definimos la regresión logística con iteraciones suficientes para converger
logreg = LogisticRegression(
    max_iter=1000,     # aumenta el límite de iteraciones para asegurar convergencia
    solver="lbfgs",    # solver robusto para problemas multiclase y datos densos
    n_jobs=-1          # paraleliza internamente algunas operaciones (si está disponible)
)

# Construimos el pipeline completo
pipe_logreg = Pipeline(steps=[
    ("fe", feature_engineering),  # añade features ingenierizadas a X
    ("pre", preprocessor),        # imputación, escalado, one-hot
    ("logreg", logreg)            # modelo de regresión logística
])

# Definimos esquema de validación cruzada estratificada (mantiene proporción 0/1 en cada fold)
cv = StratifiedKFold(
    n_splits=5,       # 5 folds
    shuffle=True,     # barajar antes de dividir
    random_state=42   # reproducibilidad
)

# Ejecutamos cross-validation con accuracy (métrica de la competición)
cv_scores = cross_val_score(
    estimator=pipe_logreg,  # el pipeline completo
    X=X,                    # features base (la FE se aplica dentro del pipeline)
    y=y,                    # target
    cv=cv,                  # esquema de validación
    scoring="accuracy",     # métrica
    n_jobs=-1               # usar todos los cores disponibles
)

# Mostramos resultados por fold y resumen estadístico
print("Accuracy por fold:", np.round(cv_scores, 4))                  # accuracies individuales
print("Accuracy promedio (CV):", cv_scores.mean().round(4),          # promedio
      "| Desv. std:", cv_scores.std().round(4))                      # dispersión entre folds


Accuracy por fold: [0.8324 0.8146 0.8146 0.8371 0.8258]
Accuracy promedio (CV): 0.8249 | Desv. std: 0.0091


# 📤 Celda 8 — Entrenamiento final y creación de `submission.csv`
Entrenamos el pipeline completo en **todo el set de entrenamiento** y generamos el archivo `submission.csv`
con las columnas `PassengerId` y `Survived` (formato requerido por Kaggle).


In [10]:
# 1) Ajustamos el pipeline completo en TODO el train
pipe_logreg.fit(X, y)                                # entrena FE + preprocesamiento + modelo

# 2) Realizamos predicciones sobre el set de test
test_preds = pipe_logreg.predict(X_test)             # obtiene 0/1 por pasajero
test_preds = test_preds.astype(int)                  # aseguramos tipo entero

# 3) Construimos el DataFrame de submission
submission = pd.DataFrame({
    "PassengerId": test_passenger_id,               # IDs del test
    "Survived": test_preds                          # predicciones binarias
})

# 4) Validaciones de formato (defensivas)
assert submission.shape[0] == 418, "El submission debe tener exactamente 418 filas."
assert list(submission.columns) == ["PassengerId", "Survived"], "Las columnas deben ser PassengerId y Survived."

# 5) Guardamos el CSV para subir a Kaggle
submission.to_csv("submission.csv", index=False)

# 6) Mensaje de confirmación y breve vista previa
print("✅ Archivo 'submission.csv' creado con", submission.shape[0], "filas.")
print(submission.head())
print("\nConteo predicciones:", submission["Survived"].value_counts().to_dict())


✅ Archivo 'submission.csv' creado con 418 filas.
   PassengerId  Survived
0          892         0
1          893         1
2          894         0
3          895         0
4          896         1

Conteo predicciones: {0: 251, 1: 167}


# 🏆 Celda 9 — Registro del resultado

- **Score Kaggle:** 0.77272  
- **Notebook:** EPA_TITANIC_LOGREG_V1.ipynb  
- **Modelo:** Regresión Logística con Feature Engineering  
- **Notas:**  
  - CV promedio (5 folds): 0.8249  
  - Generalizó mejor que KNN (subió de 0.763 → 0.772).  
  - Buena estabilidad entre folds (std = 0.0091).  
  - Primer modelo que mejora de forma consistente en leaderboard.  
